In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./pr01.csv')
NODE = data['NODE'].iloc
XCOORD, YCOORD = data['XCOORD'].iloc, data['YCOORD'].iloc
START, DEADLINE = data['START'].iloc, data['DEADLINE'].iloc
SERVICETIME, DEMAND = data['SERVICETIME'].iloc, data['DEMAND'].iloc
print(data)

    NODE  XCOORD  YCOORD  SERVICETIME  DEMAND  START  DEADLINE
0      0  -1.044   2.000            0       0      0      1440
1      1  -2.973   6.414           10       1      0      1440
2      2  -3.066   0.546           10       1      0      1440
3      3   5.164   0.547           10       1      0      1440
4      4  -1.317   6.934           10       1      0      1440
5      5  -6.741   6.832           10       1      0      1440
6      6   4.891   0.627           10       1      0      1440
7      7   0.524   2.226           10       1      0      1440
8      8  -6.500   7.723           10       1      0      1440
9      9  -0.417  -0.157           10       1      0      1440
10    10   2.303   1.164           10       1      0      1440
11    11   2.548   0.629           10       1      0      1440
12    12  -4.261  -2.639           10       1      0      1440
13    13  -7.667   9.934           10       1    325       358
14    14  -2.067   5.789           10       1    111   

In [3]:
vehicle_num = 3
vehicle_capacity = 6
customer_num = int(NODE[-1]/2)
max_duration = 480 # The route duration limit
max_ride_time = 90 # The maximum user ride time
end_system_time = 1440

In [4]:
def Dist(i, j):
    dist = np.sqrt((XCOORD[i]-XCOORD[j])**2 + (YCOORD[i]-YCOORD[j])**2)
    dist = round(dist, 2)
    return dist

def TotalDist(route):
    total_dist = 0
    for node_idx in range(len(route)-1):
        i, j = route[node_idx], route[node_idx+1]
        dist = Dist(i,j)
        total_dist += dist
    total_dist = round(total_dist, 2)
    return total_dist

In [5]:
def IsCapacityReasonable(route):
    on_board = 0
    for node in route[1:-1]:
        if node <= customer_num:
            on_board += 1
        else:
            on_board -= 1
        if on_board > vehicle_capacity:
            return False
    return True

def IsTimeReasonable(route, begin_j=0):
    for node in range(len(route)-1):
        begin_i = begin_j
        i, j = route[node], route[node+1]
        arrive_time = begin_i+SERVICETIME[i]+Dist(i,j)
        if (arrive_time > DEADLINE[j]):
            return False
        begin_j = max(START[j], arrive_time)
    return True

In [6]:
def BeginTime(route):
    begin_time = 0
    for node in route:
        if (START[node] == 0) and (DEADLINE[node] == end_system_time):
            continue
        else:
            node_idx = route.index(node)
            constrain_time = DEADLINE[node]
            end_i = constrain_time + SERVICETIME[node]
            for idx in range(node_idx, 0, -1):
                end_j = end_i
                i, j = route[idx-1], route[idx]
                end_i = end_j-SERVICETIME[j]-Dist(i,j)
            end_i = round(end_i, 2)
            reasonable = IsTimeReasonable(route[:node_idx+1], end_i)
            if reasonable == False:
                break
        begin_time = end_i
    return begin_time

def EndTime(route):
    begin_j = BeginTime(route)
    for node in range(len(route)-1):
        begin_i = begin_j
        i, j = route[node], route[node+1]
        arrive_time = begin_i+SERVICETIME[i]+Dist(i,j)
        begin_j = max(START[j], arrive_time)
    end_time = round(begin_j, 2)
    return end_time

def TotalDuration(route):
    total_duration = round(EndTime(route) - BeginTime(route), 2)
    return total_duration

def AriveTimeRecord(route):
    begin_record = []
    begin_j = BeginTime(route)
    for node in range(len(route)-1):
        begin_i = begin_j
        i, j = route[node], route[node+1]
        arrive_time = begin_i+SERVICETIME[i]+Dist(i,j)
        begin_j = max(START[j], arrive_time)
        begin_record.append(round(begin_j, 2))
    return begin_record

In [7]:
def IsRideTimeReasonable(route, pick_node_idx, deliver_node_idx):
    record = AriveTimeRecord(route)
    ride_time = record[deliver_node_idx] - record[pick_node_idx]
    if ride_time > max_ride_time:
        return False
    else:
        return True

In [8]:
def RouteCost(route):
    cost = TotalDist(route)
    return cost

In [9]:
def ComputeRegret(rqst, rt):
    origin_cost = RouteCost(rt)
    pick_node, deliver_node = rqst, rqst+customer_num
    regret_value = 9999
    best_new_rt = rt
    for i in range(1, len(rt)):
        for j in range(i, len(rt)):
            new_rt = rt[:i] + [pick_node] + rt[i:j] + [deliver_node] + rt[j:]
            pick_idx, deliver_idx = new_rt.index(pick_node), new_rt.index(deliver_node)
            if (IsCapacityReasonable(new_rt)==False) \
                or (IsTimeReasonable(new_rt, BeginTime(new_rt))==False) \
                or (IsRideTimeReasonable(new_rt, pick_idx, deliver_idx)==False) \
                or (TotalDuration(new_rt) > max_duration):
                continue
            new_cost = RouteCost(new_rt)
            regret = new_cost - origin_cost
            if regret < regret_value:
                regret_value = regret
                best_new_rt = new_rt
    regret_value = round(regret_value, 2)
    return regret_value, best_new_rt

In [10]:
initial = [[0] for _ in range(vehicle_num)]
earlier_node = []
for customer in range(1, customer_num+1):
    if DEADLINE[customer] != end_system_time:
        earlier_node.append([NODE[customer], START[customer]])
earlier_node.sort(key=lambda x: x[1])
add_idx = 0 # k
for i in range(vehicle_num):
    initial[i].append(earlier_node[add_idx][0])
    initial[i].append(earlier_node[add_idx][0] + customer_num)
    initial[i].append(0)
    while True:
        pick_node_id = earlier_node[add_idx][0] # point k
        next_pick_node_id = earlier_node[add_idx+1][0] # point k+1
        deliver_node_id = pick_node_id + customer_num
        add_idx += 1
        # verified LDT_k + TT_D(k),P(k+1) <= EPT(k+1)
        LDT = DEADLINE[deliver_node_id]
        TT  = Dist(deliver_node_id, next_pick_node_id)
        EPT = START[next_pick_node_id]
        if LDT + TT > EPT:
            break
print(initial)

[[0, 17, 41, 0], [0, 14, 38, 0], [0, 22, 46, 0]]


In [11]:
unselected = [i for i in range(1,customer_num+1)]
for route in initial:
    for node in route:
        if node in unselected:
            unselected.remove(node)

while unselected:
    final_route = initial
    incremental_cost_matrix = np.zeros((len(unselected), vehicle_num))
    route_matrix = np.empty((len(unselected), vehicle_num), dtype=object)
    for request_idx, request in enumerate(unselected):
        for route_idx, route in enumerate(final_route):
            regret_value, inserted_route = ComputeRegret(request, route)
            incremental_cost_matrix[request_idx, route_idx] = regret_value
            route_matrix[request_idx, route_idx] = inserted_route
    min_value = incremental_cost_matrix.min(axis=1).reshape(-1, 1)
    incremental_cost_matrix = incremental_cost_matrix - min_value
    sum_matrix = incremental_cost_matrix.sum(axis=1)
    best_insert_row_idx = np.argmax(sum_matrix)
    best_insert_column_idx = np.argmin(incremental_cost_matrix[best_insert_row_idx])
    final_route[best_insert_column_idx] = route_matrix[best_insert_row_idx][best_insert_column_idx]
    unselected.remove(unselected[best_insert_row_idx])

In [12]:
total_cost = 0
total_duration = 0
for idx, route in enumerate(final_route):
    total_cost += RouteCost(route)
    total_duration += TotalDuration(route)
    print(f"Route {idx}: {route}")
print("Total Distance:", round(total_cost, 2))
print("Total Duration:", round(total_duration, 2))

Route 0: [0, 9, 17, 8, 33, 20, 1, 41, 44, 32, 2, 25, 26, 13, 4, 37, 28, 19, 23, 47, 43, 0]
Route 1: [0, 14, 5, 29, 16, 18, 40, 38, 21, 42, 45, 0]
Route 2: [0, 7, 22, 11, 3, 46, 35, 31, 27, 10, 34, 24, 12, 48, 6, 36, 15, 30, 39, 0]
Total Distance: 195.89
Total Duration: 1098.19
